In [1]:
import sys
from PyQt5.QtWidgets import *
from PyQt5.QtGui import *
from PyQt5.QtCore import *
from PyQt5.QtCore import Qt
import matplotlib.pyplot as plt
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib import font_manager, rc
import requests
import re
import json
import pandas as pd
from pandas.io.json import json_normalize
from IPython.display import display_html
import numpy as np
from bs4 import BeautifulSoup

def getFStatementsFromNaverFinance(iCode=None):
    #iCode = '005930'
    # iCode = '000660'
    first_url = 'http://companyinfo.stock.naver.com/v1/company/c1010001.aspx?cmp_cd=' + str(iCode)
    print(first_url)
    first_html_text = requests.get(first_url).text
    #print(first_html_text)

    encparam = re.findall("encparam: '(.*?)'", first_html_text)[0]
    print(encparam)
    id = re.findall("id: '(.*?)'", first_html_text)[0]
    print(id)
    ##########################################################################################


    soup = BeautifulSoup(first_html_text, "lxml")
    res = soup.find('table', attrs={'class': 'gHead'})
    # print(str(res))
    small = pd.read_html(str(res))
    small = small[0]
    # display_html(str(res), raw=True)

    # df = df[0]
    # \print(res)
    # result = soup.find('table', attrs={'summary':'시가총액 정보'})
    # print(result)
    # 시총 = soup.find(string="시가총액")
    # print(시총)

    ##########################################################################################
    # headers={'Accept': 'application/json, text/javascript, */*; q=0.01','Referer': 'http://companyinfo.stock.naver.com'}
    #headers ={'Accept': 'application/json'}
    header = {
    'Referer': 'https://companyinfo.stock.naver.com/v1/company/c1010001.aspx',
    }
    second_url = 'http://companyinfo.stock.naver.com/v1/company/ajax/cF1001.aspx?cmp_cd=' + str(iCode) + '&fin_typ=0&freq_typ=Y&encparam=' + encparam + '&id=' + id
    print(second_url)
    second_html_text = requests.get(second_url, headers=header).text
    #second_html_text = requests.get(second_url, headers={'Accept': 'application/json'}).text
    #second_html_text = requests.get(second_url).text
    #display_html(second_html_text, raw=True)
    #print('result :' + second_html_text)

    soup = BeautifulSoup(second_html_text, "lxml")
    # td = soup.find('table', attrs={'class':'gHead01 all-width'})
    # table = soup.find('div', attrs={'id':'bG05RlB6cn'})
    td = soup.find('table', attrs={'class': 'gHead01 all-width'})
    #print('len = '+str(len(td)))
    # bG05RlB6cn
    ##########################################################################################
    # print(second_html_text)
    # print(str(td))

    ###display_html(str(td), raw=True)

    #print(str(td))
    # df = pd.read_html(str(td), skiprows=1)
    df = pd.read_html(str(td))
    df = df[0]
    # print(df.dtypes)

    temp_col = ['구분']
    # for i in range(8):
    #    temp_col.append(df.columns[i][1])

    for i in range(8):
        result = re.search("[0-9]*/[0-9]*", str(df.columns[i][1]))
        data_str = result.group()
        data_str = data_str.replace('/', '-')
        temp_col.append(data_str)
    # print(data_str)
    df.columns = temp_col
    df = df.set_index('구분')
    df = df.fillna(0)
    df.loc['발행주식수(보통주)'] = df.loc['발행주식수(보통주)'].astype(np.int64)
    df = df.T
    df.index = pd.to_datetime(df.index)
    return df

In [2]:
kospi200 = [['005930', '삼성전자'], ['000660', 'SK하이닉스'], ['028260', '삼성물산'], ['015760', '한국전력'], ['005490', 'POSCO'], ['012330', '현대모비스'], ['105560', 'KB금융'], ['032830', '삼성생명'], ['017670', 'SK텔레콤'], ['051910', 'LG화학'], ['034730', 'SK'], ['207940', '삼성바이오로직스'], ['090430', '아모레퍼시픽'], ['033780', 'KT&G'], ['051900', 'LG생활건강'], ['096770', 'SK이노베이션'], ['086790', '하나금융지주'], ['003550', 'LG'], ['000810', '삼성화재'], ['034220', 'LG디스플레이'], ['066570', 'LG전자'], ['000030', '우리은행'], ['006400', '삼성SDI'], ['011170', '롯데케미칼'], ['002790', '아모레G'], ['010950', 'S-Oil'], ['009540', '현대중공업'], ['023530', '롯데쇼핑'], ['010130', '고려아연'], ['030200', 'KT'], ['036570', '엔씨소프트'], ['004020', '현대제철'], ['024110', '기업은행'], ['161390', '한국타이어'], ['021240', '코웨이'], ['009150', '삼성전기'], ['035250', '강원랜드'], ['006800', '미래에셋대우'], ['032640', 'LG유플러스'], ['139480', '이마트'], ['078930', 'GS'], ['088350', '한화생명'], ['086280', '현대글로비스'], ['004800', '효성'], ['047810', '한국항공우주'], ['001040', 'CJ'], ['079160', 'CJ'], ['018880', '한온시스템'], ['008930', '한미사이언스'], ['000720', '현대건설'], ['282330', 'BGF리테일'], ['009830', '한화케미칼'], ['036460', '한국가스공사'], ['010140', '삼성중공업'], ['097950', 'CJ제일제당'], ['097955', 'CJ제일제당'], ['002380', 'KCC'], ['029780', '삼성카드'], ['009240', '한샘'], ['005940', 'NH투자증권'], ['128940', '한미약품'], ['000120', 'CJ대한통운'], ['007070', 'GS리테일'], ['071050', '한국금융지주'], ['011070', 'LG이노텍'], ['016360', '삼성증권'], ['012750', '에스원'], ['003490', '대한항공'], ['138930', 'BNK금융지주'], ['012630', '현대산업'], ['001450', '현대해상'], ['047040', '대우건설'], ['000210', '대림산업'], ['008560', '메리츠종금증권'], ['000100', '유한양행'], ['047050', '포스코대우'], ['007310', '오뚜기'], ['069960', '현대백화점'], ['060980', '만도'], ['204320', '만도'], ['000150', '두산'], ['006260', 'LS'], ['012450', '한화테크윈'], ['028050', '삼성엔지니어링'], ['004170', '신세계'], ['035510', '신세계'], ['008770', '호텔신라'], ['011780', '금호석유'], ['034020', '두산중공업'], ['010620', '현대미포조선'], ['006360', 'GS건설'], ['007070', 'GS리테일'], ['073240', '금호타이어'], ['007310', '오뚜기'], ['010620', '현대미포조선'], ['006280', '녹십자'], ['105630', '한세실업'], ['108670', 'LG하우시스'], ['010780', '아이에스동서'], ['003240', '태광산업'], ['001430', '세아베스틸'], ['049770', '동원F&B'], ['002350', '넥센타이어'], ['120110', '코오롱인더'], ['161890', '한국콜마'], ['001120', 'LG상사'], ['069260', '휴켐스'], ['003620', '쌍용차'], ['003300', '한일시멘트'], ['011790', 'SKC'], ['003410', '쌍용양회'], ['002270', '롯데푸드'], ['033920', '무학'], ['128940', '한미약품'], ['069620', '대웅제약'], ['002240', '고려제강'], ['005180', '빙그레'], ['020000', '한섬'], ['029530', '신도리코'], ['000070', '삼양홀딩스'], ['170900', '동아에스티'], ['005850', '에스엘'], ['025540', '한국단자'], ['064960', 'S&T모티브'], ['115390', '락앤락'], ['000640', '동아쏘시오홀딩스'], ['009290', '광동제약'], ['002960', '한국쉘석유'], ['003000', '부광약품'], ['001230', '동국제강'], ['004490', '세방전지'], ['003030', '세아제강'], ['006650', '대한유화'], ['000480', '조선내화'], ['003920', '남양유업'], ['034120', 'SBS'], ['097230', '한진중공업'], ['007570', '일양약품'], ['282690', '동아타이어'], ['082740', '두산엔진'], ['008060', '대덕전자'], ['079980', '휴비스'], ['009680', '모토닉'], ['213500', '한솔제지'], ['025860', '남해화학'], ['000430', '대원강업'], ['014830', '유니드'], ['016800', '퍼시스'], ['005090', '삼광글라스'], ['020150', '일진머티리얼즈'], ['009580', '무림P&P'], ['003570', 'S&T중공업'], ['033530', '세종공업'], ['104700', '한국철강'], ['078520', '에이블씨엔씨'], ['010690', '화신'], ['002020', '코오롱'], ['000140', '하이트진로홀딩스'], ['007690', '국도화학'], ['003520', '영진약품'], ['004130', '대덕GDS'], ['093370', '후성'], ['004710', '한솔테크닉스'], ['033240', '자화전자'], ['020760', '일진디스플'], ['007810', '코리아써키트'], ['004430', '송원산업'], ['005950', '이수화학'], ['006380', '카프로']]


In [3]:
getFStatementsFromNaverFinance('005930')


http://companyinfo.stock.naver.com/v1/company/c1010001.aspx?cmp_cd=005930
bjFLTW41ZUhaTEdzZXFzakd4Q1dJdz09
ZTRzQlVCd0
http://companyinfo.stock.naver.com/v1/company/ajax/cF1001.aspx?cmp_cd=005930&fin_typ=0&freq_typ=Y&encparam=bjFLTW41ZUhaTEdzZXFzakd4Q1dJdz09&id=ZTRzQlVCd0


구분,매출액,영업이익,영업이익(발표기준),세전계속사업이익,당기순이익,당기순이익(지배),당기순이익(비지배),자산총계,부채총계,자본총계,...,부채비율,자본유보율,EPS(원),PER(배),BPS(원),PBR(배),현금DPS(원),현금배당수익률,현금배당성향(%),발행주식수(보통주)
2013-12-01,2286927.0,367850.0,367850.0,383643.0,304748.0,298212.0,6535.0,2140750.0,640590.0,1500160.0,...,42.70,17047.55,3506.0,8.27,19161.0,1.51,286.0,0.99,7.23,7.364967e+09
2014-12-01,2062060.0,250251.0,250251.0,278750.0,233944.0,230825.0,3119.0,2304230.0,623348.0,1680882.0,...,37.09,19379.47,2713.0,10.33,21664.0,1.29,400.0,1.43,13.00,7.364967e+09
2015-12-01,2006535.0,264134.0,264134.0,259610.0,190601.0,186946.0,3655.0,2421795.0,631197.0,1790598.0,...,35.25,21117.88,2198.0,12.11,23715.0,1.12,420.0,1.58,16.42,7.364967e+09
2016-12-01,2018667.0,292407.0,292407.0,307137.0,227261.0,224157.0,3104.0,2621743.0,692113.0,1929630.0,...,35.87,22004.14,2735.0,13.91,26636.0,1.43,570.0,1.50,17.81,7.033967e+09
2017-12-01,2395754.0,536450.0,536450.0,561960.0,421867.0,413446.0,8422.0,3017521.0,872607.0,2144914.0,...,40.68,24536.12,5421.0,9.93,30427.0,1.77,850.0,1.58,14.09,6.454925e+09
2018-12-01,2488558.0,625939.0,0.0,646532.0,471875.0,465929.0,0.0,3443631.0,915198.0,2528433.0,...,36.20,0.00,6395.0,6.09,36101.0,1.08,1413.0,3.63,18.10,0.000000e+00
2019-12-01,2467463.0,536642.0,0.0,563363.0,412469.0,406735.0,0.0,3868013.0,918412.0,2826874.0,...,32.49,0.00,5988.0,6.50,40354.0,0.97,1487.0,3.82,21.82,0.000000e+00
2020-12-01,2568656.0,578298.0,0.0,609092.0,445916.0,439439.0,0.0,4117597.0,973265.0,3144332.0,...,30.95,0.00,6469.0,6.02,44961.0,0.87,1643.0,4.22,22.32,0.000000e+00


In [94]:

for i in range(10) : 
    #print( kospi200[i][0] )
    df = getFStatementsFromNaverFinance(str(kospi200[i][0]))
    display_html(df)
#df

http://companyinfo.stock.naver.com/v1/company/c1010001.aspx?cmp_cd=005930
d0kwdGw4VTlJMVh0NVg2Mm9ZVEZlUT09
VGVTbkwxZ2
http://companyinfo.stock.naver.com/v1/company/ajax/cF1001.aspx?cmp_cd=005930&fin_typ=0&freq_typ=Y&encparam=d0kwdGw4VTlJMVh0NVg2Mm9ZVEZlUT09&id=VGVTbkwxZ2


구분,매출액,영업이익,영업이익(발표기준),세전계속사업이익,당기순이익,당기순이익(지배),당기순이익(비지배),자산총계,부채총계,자본총계,...,부채비율,자본유보율,EPS(원),PER(배),BPS(원),PBR(배),현금DPS(원),현금배당수익률,현금배당성향(%),발행주식수(보통주)
2013-12-01,2286927.0,367850.0,367850.0,383643.0,304748.0,298212.0,6535.0,2140750.0,640590.0,1500160.0,...,42.70,17047.55,3506.0,7.83,19161.0,1.43,286.0,1.04,7.23,7.364967e+09
2014-12-01,2062060.0,250251.0,250251.0,278750.0,233944.0,230825.0,3119.0,2304230.0,623348.0,1680882.0,...,37.09,19379.47,2713.0,9.78,21664.0,1.23,400.0,1.51,13.00,7.364967e+09
2015-12-01,2006535.0,264134.0,264134.0,259610.0,190601.0,186946.0,3655.0,2421795.0,631197.0,1790598.0,...,35.25,21117.88,2198.0,11.47,23715.0,1.06,420.0,1.67,16.42,7.364967e+09
2016-12-01,2018667.0,292407.0,292407.0,307137.0,227261.0,224157.0,3104.0,2621743.0,692113.0,1929630.0,...,35.87,22004.14,2735.0,13.18,26636.0,1.35,570.0,1.58,17.81,7.033967e+09
2017-12-01,2395754.0,536450.0,536450.0,561960.0,421867.0,413446.0,8422.0,3017521.0,872607.0,2144914.0,...,40.68,24536.12,5421.0,9.40,30427.0,1.67,850.0,1.67,14.09,6.454925e+09
2018-12-01,2505079.0,640765.0,0.0,663057.0,484099.0,478182.0,0.0,3446406.0,912944.0,2533461.0,...,36.04,0.00,6563.0,6.24,38745.0,1.06,1502.0,3.67,20.16,0.000000e+00
2019-12-01,2533047.0,590437.0,0.0,624354.0,457140.0,451675.0,0.0,3911993.0,915212.0,2874055.0,...,31.84,0.00,6649.0,6.16,44018.0,0.93,1496.0,3.65,21.27,0.000000e+00
2020-12-01,2646046.0,643589.0,0.0,676420.0,495147.0,487731.0,0.0,4208307.0,955474.0,3252832.0,...,29.37,0.00,7180.0,5.70,49910.0,0.82,1561.0,3.81,20.54,0.000000e+00


http://companyinfo.stock.naver.com/v1/company/c1010001.aspx?cmp_cd=000660
SHpBa2sxdDJEZnRweXFSajdBekt4Zz09
ZTRzQlVCd0
http://companyinfo.stock.naver.com/v1/company/ajax/cF1001.aspx?cmp_cd=000660&fin_typ=0&freq_typ=Y&encparam=SHpBa2sxdDJEZnRweXFSajdBekt4Zz09&id=ZTRzQlVCd0


구분,매출액,영업이익,영업이익(발표기준),세전계속사업이익,당기순이익,당기순이익(지배),당기순이익(비지배),자산총계,부채총계,자본총계,...,부채비율,자본유보율,EPS(원),PER(배),BPS(원),PBR(배),현금DPS(원),현금배당수익률,현금배당성향(%),발행주식수(보통주)
2013-12-01,141651.0,33798.0,33798.0,30749.0,28729.0,28725.0,4.0,207973.0,77304.0,130669.0,...,59.16,269.22,4099.0,8.98,18399.0,2.00,0.0,0.00,0.00,710200891.0
2014-12-01,171256.0,51095.0,51095.0,50477.0,41952.0,41955.0,-3.0,268833.0,88470.0,180363.0,...,49.05,394.26,5842.0,8.17,24775.0,1.93,300.0,0.63,5.21,728002365.0
2015-12-01,187980.0,53361.0,53361.0,52691.0,43236.0,43224.0,12.0,296779.0,82902.0,213877.0,...,38.76,505.86,5937.0,5.18,30293.0,1.02,500.0,1.63,8.17,728002365.0
2016-12-01,171980.0,32767.0,32767.0,32165.0,29605.0,29538.0,67.0,322160.0,81925.0,240235.0,...,34.10,579.89,4057.0,11.02,34018.0,1.31,600.0,1.34,14.34,728002365.0
2017-12-01,301094.0,137213.0,137213.0,134396.0,106422.0,106415.0,7.0,454185.0,115975.0,338209.0,...,34.29,859.32,14617.0,5.23,47897.0,1.60,1000.0,1.31,6.63,728002365.0
2018-12-01,417606.0,222683.0,0.0,224152.0,165049.0,165377.0,0.0,621847.0,129757.0,492089.0,...,26.37,0.00,22717.0,2.94,71950.0,0.93,1480.0,2.22,6.52,0.0
2019-12-01,415021.0,196073.0,0.0,198427.0,145924.0,147040.0,0.0,760612.0,133467.0,627145.0,...,21.28,0.00,20198.0,3.31,91616.0,0.73,1743.0,2.61,8.63,0.0
2020-12-01,452998.0,213954.0,0.0,213223.0,158455.0,157785.0,0.0,912363.0,140558.0,771806.0,...,18.21,0.00,21674.0,3.08,112893.0,0.59,1909.0,2.86,8.81,0.0


http://companyinfo.stock.naver.com/v1/company/c1010001.aspx?cmp_cd=028260
S3VYbGVlNzc3eDkxUzdhL0gwQmg1dz09
ZlEwemUxRm
http://companyinfo.stock.naver.com/v1/company/ajax/cF1001.aspx?cmp_cd=028260&fin_typ=0&freq_typ=Y&encparam=S3VYbGVlNzc3eDkxUzdhL0gwQmg1dz09&id=ZlEwemUxRm


...

구분,매출액,영업이익,영업이익(발표기준),세전계속사업이익,당기순이익,당기순이익(지배),당기순이익(비지배),자산총계,부채총계,자본총계,...,부채비율,자본유보율,EPS(원),PER(배),BPS(원),PBR(배),현금DPS(원),현금배당수익률,현금배당성향(%),발행주식수(보통주)
2013-12-01,193044.0,5426.0,5426.0,7585.0,5844.0,5870.0,-25.0,1929394.0,1739090.0,190304.0,...,913.85,19809.14,2935.0,35.44,99281.0,1.05,850.0,0.82,27.67,200000000.0
2014-12-01,274264.0,14055.0,14055.0,16035.0,13370.0,13375.0,-5.0,2144337.0,1921490.0,222847.0,...,862.25,23265.02,6687.0,17.42,117440.0,0.99,1800.0,1.55,25.45,200000000.0
2015-12-01,277059.0,11485.0,11485.0,13715.0,12096.0,12112.0,-17.0,2303629.0,2066412.0,237217.0,...,871.10,25190.12,6056.0,18.16,128139.0,0.86,1800.0,1.64,27.48,200000000.0
2016-12-01,304286.0,9865.0,9865.0,26075.0,21500.0,20543.0,957.0,2646538.0,2363454.0,283084.0,...,834.89,30318.82,10271.0,10.95,148374.0,0.76,1200.0,1.07,10.49,200000000.0
2017-12-01,319590.0,16906.0,16906.0,16828.0,12632.0,11661.0,971.0,2827138.0,2515922.0,311216.0,...,808.42,33131.94,5831.0,21.35,163491.0,0.76,2000.0,1.61,30.80,200000000.0
2018-12-01,163894.0,13825.0,0.0,28304.0,20523.0,19988.0,0.0,2932244.0,2612412.0,318983.0,...,818.98,0.00,9994.0,8.53,164470.0,0.52,3003.0,3.53,30.05,0.0
2019-12-01,163884.0,5113.0,0.0,19758.0,14466.0,13837.0,0.0,3053233.0,2719939.0,332056.0,...,819.12,0.00,6918.0,12.32,167873.0,0.51,2309.0,2.71,33.38,0.0
2020-12-01,168538.0,6166.0,0.0,21107.0,15295.0,14804.0,0.0,3181808.0,2831067.0,348563.0,...,812.21,0.00,7402.0,11.51,172395.0,0.49,2426.0,2.85,32.78,0.0


http://companyinfo.stock.naver.com/v1/company/c1010001.aspx?cmp_cd=017670
NHQ5U2tBbTkrVlpraVBJaURpZlhLdz09
TDdCYnFkT0
http://companyinfo.stock.naver.com/v1/company/ajax/cF1001.aspx?cmp_cd=017670&fin_typ=0&freq_typ=Y&encparam=NHQ5U2tBbTkrVlpraVBJaURpZlhLdz09&id=TDdCYnFkT0


구분,매출액,영업이익,영업이익(발표기준),세전계속사업이익,당기순이익,당기순이익(지배),당기순이익(비지배),자산총계,부채총계,자본총계,...,부채비율,자본유보율,EPS(원),PER(배),BPS(원),PBR(배),현금DPS(원),현금배당수익률,현금배당성향(%),발행주식수(보통주)
2013-12-01,166021.0,20111.0,20111.0,18271.0,16095.0,16390.0,-294.0,265765.0,124100.0,141666.0,...,87.60,35884.28,20298.0,11.33,189640.0,1.21,9400.0,4.09,40.66,80745711.0
2014-12-01,171638.0,18251.0,18251.0,22538.0,17993.0,18012.0,-19.0,279412.0,126930.0,152483.0,...,83.24,38317.34,22307.0,12.01,204504.0,1.31,9400.0,3.51,37.02,80745711.0
2015-12-01,171367.0,17080.0,17080.0,20354.0,15159.0,15186.0,-27.0,285814.0,132073.0,153741.0,...,85.91,40152.14,18807.0,11.46,215993.0,1.00,10000.0,4.64,46.63,80745711.0
2016-12-01,170918.0,15357.0,15357.0,20961.0,16601.0,16760.0,-159.0,312977.0,151812.0,161164.0,...,94.20,42270.33,20756.0,10.79,226194.0,0.99,10000.0,4.46,42.13,80745711.0
2017-12-01,175200.0,15366.0,15366.0,34032.0,26576.0,25998.0,578.0,334287.0,153995.0,180292.0,...,85.41,46488.12,32198.0,8.29,252689.0,1.06,10000.0,3.75,27.16,80745711.0
2018-12-01,168994.0,13138.0,0.0,43952.0,34487.0,33979.0,0.0,369629.0,152987.0,216642.0,...,70.62,0.00,42081.0,6.73,302423.0,0.94,10125.0,3.58,24.06,0.0
2019-12-01,174151.0,14528.0,0.0,42609.0,33086.0,32589.0,0.0,397514.0,155116.0,242398.0,...,63.99,0.00,40360.0,7.01,338214.0,0.84,10300.0,3.64,25.52,0.0
2020-12-01,180144.0,15984.0,0.0,43290.0,33689.0,33060.0,0.0,423463.0,154809.0,268655.0,...,57.62,0.00,40943.0,6.91,374604.0,0.76,10400.0,3.67,25.40,0.0


http://companyinfo.stock.naver.com/v1/company/c1010001.aspx?cmp_cd=051910
QlBPUFkxOHBaZnRQa2ZuWFB3am4yQT09
aFVlanREZS
http://companyinfo.stock.naver.com/v1/company/ajax/cF1001.aspx?cmp_cd=051910&fin_typ=0&freq_typ=Y&encparam=QlBPUFkxOHBaZnRQa2ZuWFB3am4yQT09&id=aFVlanREZS


구분,매출액,영업이익,영업이익(발표기준),세전계속사업이익,당기순이익,당기순이익(지배),당기순이익(비지배),자산총계,부채총계,자본총계,...,부채비율,자본유보율,EPS(원),PER(배),BPS(원),PBR(배),현금DPS(원),현금배당수익률,현금배당성향(%),발행주식수(보통주)
2013-12-01,231436.0,17430.0,17430.0,16013.0,12706.0,12660.0,46.0,174465.0,57208.0,117257.0,...,48.79,3066.41,17131.0,17.48,157707.0,1.90,4000.0,1.34,23.26,66271100.0
2014-12-01,225778.0,13108.0,13108.0,11599.0,8540.0,8679.0,-139.0,181276.0,58618.0,122659.0,...,47.79,3206.60,11745.0,15.41,165091.0,1.10,4000.0,2.21,33.93,66271100.0
2015-12-01,202066.0,18236.0,18236.0,15496.0,11485.0,11530.0,-45.0,185787.0,54752.0,131035.0,...,41.78,3434.44,15602.0,21.06,176671.0,1.86,4500.0,1.37,28.73,66271100.0
2016-12-01,206593.0,19919.0,19919.0,16598.0,12810.0,12811.0,-1.0,204871.0,64361.0,140510.0,...,45.80,3686.21,17336.0,15.06,192955.0,1.35,5000.0,1.92,28.73,66271100.0
2017-12-01,256980.0,29285.0,29285.0,25639.0,20220.0,19453.0,767.0,250412.0,87026.0,163386.0,...,53.26,4167.96,24854.0,16.30,211043.0,1.92,6000.0,1.48,23.65,70592343.0
2018-12-01,280150.0,24067.0,0.0,21746.0,16979.0,16488.0,0.0,279763.0,103951.0,175811.0,...,59.13,0.00,21062.0,16.69,226668.0,1.55,6268.0,1.78,26.84,0.0
2019-12-01,322554.0,25816.0,0.0,23433.0,18165.0,17789.0,0.0,304986.0,115988.0,189000.0,...,61.37,0.00,22724.0,15.47,243221.0,1.45,6398.0,1.82,25.39,0.0
2020-12-01,371987.0,31186.0,0.0,28688.0,22211.0,21698.0,0.0,333281.0,127364.0,205917.0,...,61.85,0.00,27718.0,12.68,264330.0,1.33,7147.0,2.03,23.25,0.0


In [3]:
getFStatementsFromNaverFinance('282330')

http://companyinfo.stock.naver.com/v1/company/c1010001.aspx?cmp_cd=282330
Y0x6dHpoKzYvc1lZZTBCc1BIWkViUT09
VGVTbkwxZ2
http://companyinfo.stock.naver.com/v1/company/ajax/cF1001.aspx?cmp_cd=282330&fin_typ=0&freq_typ=Y&encparam=Y0x6dHpoKzYvc1lZZTBCc1BIWkViUT09&id=VGVTbkwxZ2


AttributeError: 'NoneType' object has no attribute 'group'

In [12]:
import sys
from PyQt5.QtWidgets import *
from PyQt5.QtGui import *
from PyQt5.QtCore import *
from PyQt5.QtCore import Qt
import matplotlib.pyplot as plt
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib import font_manager, rc
import requests
import re
import json
import pandas as pd
from pandas.io.json import json_normalize
from IPython.display import display_html
import numpy as np
from bs4 import BeautifulSoup


def getDF(iCode=None):

    #iCode = '005930'
    # iCode = '000660'
    first_url = 'http://companyinfo.stock.naver.com/v1/company/c1010001.aspx?cmp_cd=' + str(iCode)
    print(first_url)
    first_html_text = requests.get(first_url).text
    #print(first_html_text)

    encparam = re.findall("encparam: '(.*?)'", first_html_text)[0]
    print(encparam)
    id = re.findall("id: '(.*?)'", first_html_text)[0]
    print(id)
    ##########################################################################################


    soup = BeautifulSoup(first_html_text, "lxml")
    res = soup.find('table', attrs={'class': 'gHead'})
    # print(str(res))
    small = pd.read_html(str(res))
    small = small[0]
    # display_html(str(res), raw=True)

    # df = df[0]
    # \print(res)
    # result = soup.find('table', attrs={'summary':'시가총액 정보'})
    # print(result)
    # 시총 = soup.find(string="시가총액")
    # print(시총)

    ##########################################################################################
    # headers={'Accept': 'application/json, text/javascript, */*; q=0.01','Referer': 'http://companyinfo.stock.naver.com'}
    #headers ={'Accept': 'application/json'}
    header = {
    'Referer': 'https://companyinfo.stock.naver.com/v1/company/c1010001.aspx',
    }
    second_url = 'http://companyinfo.stock.naver.com/v1/company/ajax/cF1001.aspx?cmp_cd=' + str(iCode) + '&fin_typ=0&freq_typ=Y&encparam=' + encparam + '&id=' + id
    print(second_url)
    second_html_text = requests.get(second_url, headers=header).text
    #second_html_text = requests.get(second_url, headers={'Accept': 'application/json'}).text
    #second_html_text = requests.get(second_url).text
    #display_html(second_html_text, raw=True)
    #print('result :' + second_html_text)

    soup = BeautifulSoup(second_html_text, "lxml")
    # td = soup.find('table', attrs={'class':'gHead01 all-width'})
    # table = soup.find('div', attrs={'id':'bG05RlB6cn'})
    td = soup.find('table', attrs={'class': 'gHead01 all-width'})
    #print('len = '+str(len(td)))
    # bG05RlB6cn
    ##########################################################################################
    # print(second_html_text)
    # print(str(td))

    ###display_html(str(td), raw=True)

    #print(str(td))
    # df = pd.read_html(str(td), skiprows=1)
    df = pd.read_html(str(td))
    df = df[0]
    print(df)
    #print(df.dtypes)
    

    temp_col = ['구분']
    # for i in range(8):
    #    temp_col.append(df.columns[i][1])

    for i in range(8):
        result = re.search("[0-9]*/[0-9]*", str(df.columns[i][1]))
        data_str = result.group()
        data_str = data_str.replace('/', '-')
        temp_col.append(data_str)
    # print(data_str)
    df.columns = temp_col
    df = df.set_index('구분')
    df = df.fillna(0)
    df.loc['발행주식수(보통주)'] = df.loc['발행주식수(보통주)'].astype(np.int64)
    df = df.T
    df.index = pd.to_datetime(df.index)
    return df

#iCode = '282330'
#iCode = '282690'
iCode = '213500'
getDF(iCode)

http://companyinfo.stock.naver.com/v1/company/c1010001.aspx?cmp_cd=213500
RWM2YVExRmk4b2pHMDdMUmR6aVpIUT09
TDdCYnFkT0
http://companyinfo.stock.naver.com/v1/company/ajax/cF1001.aspx?cmp_cd=213500&fin_typ=0&freq_typ=Y&encparam=RWM2YVExRmk4b2pHMDdMUmR6aVpIUT09&id=TDdCYnFkT0
               주요재무정보                연간 Unnamed: 2_level_0 Unnamed: 3_level_0  \
   Unnamed: 0_level_1 2014/12  (IFRS연결)  2015/12  (IFRS연결)  2016/12  (IFRS연결)   
0                 매출액               NaN                NaN           15117.00   
1                영업이익               NaN                NaN             750.00   
2          영업이익(발표기준)               NaN                NaN             750.00   
3            세전계속사업이익               NaN                NaN             343.00   
4               당기순이익               NaN                NaN             221.00   
5           당기순이익(지배)               NaN                NaN             241.00   
6          당기순이익(비지배)               NaN                NaN             -20.00   

AttributeError: 'NoneType' object has no attribute 'group'